# Creating Custom Groupings

We wanted the ability to compare school districts based on similar school districts as well as by statewide.  This notebook creates the groupings.  The output files for these groupings were used in the Tableau visualization.

Start with the standard imports we have used for every notebook in this class.

In [36]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

We need to use the cleaned file instead of the filtered file because we need to get some of the columns that we dropped from the filtered file.

In [37]:
districts = pd.read_csv("data/finaldata/cleaned.csv")

Apply the filtering logic that we used in the main notebook.

In [38]:
import math
print 'Total number of unique school districts: ' + str(len(np.unique(districts['Agency ID - NCES Assigned [District] Latest available year'])))
districts = districts[districts['Highest Grade Offered [District] 2009-10']=='12th Grade']
print 'Total number of school districts that have high schools: ' + str(len(districts))
districts = districts[districts['SURVYEAR']!='–']
print 'Total number of school districts that have a row on raw graduation data: ' + str(len(districts))
districts = districts[districts['AFGR']>=0]
print 'Total number of school districts with valid graduation data: ' + str(len(districts))

Total number of unique school districts: 19023
Total number of school districts that have high schools: 12955
Total number of school districts that have a row on raw graduation data: 12955
Total number of school districts with valid graduation data: 10785


In [39]:
districts=districts.replace([np.inf, -np.inf], np.nan)

Mean, min, max graduation rates by state for export to the visualization.

In [40]:
stateresults = districts.groupby('State Name [District] Latest available year')['AFGR'].aggregate([np.mean, np.min, np.max])
stateresults.to_csv("data/finaldata/stateresults.csv")
stateresults.head(100)

,mean,amin,amax
State Name [District] Latest available year,,,
Alabama,73.879389,32.8,99.7
Alaska,71.076471,23.3,100.0
Arizona,74.582390,2.0,100.0
Arkansas,78.018548,10.3,100.0
California,78.743750,7.5,100.0
Colorado,80.590286,12.5,100.0
Delaware,73.138095,45.8,100.0
District of Columbia,65.814286,48.1,87.5
Florida,69.871233,26.1,97.5


In [41]:
districts['Urban-centric Locale [District] 2009-10'].value_counts()

42-Rural: Distant      2539
43-Rural: Remote       1939
21-Suburb: Large       1586
41-Rural: Fringe       1366
32-Town: Distant       1058
33-Town: Remote         801
13-City: Small          376
11-City: Large          292
31-Town: Fringe         275
22-Suburb: Mid-size     201
23-Suburb: Small        177
12-City: Mid-size       175
dtype: int64

Calculate the ratio of free and reduced lunch students to total students.

In [42]:
districts['r_lunch_free_reduced'] = districts['Total Free and Reduced Lunch Students [Public School] 2009-10']/districts['Total Students [Public School] 2009-10']

Determine the top quartile and bottom quartile boundaries for Large and Small school size.

In [43]:
topquartile = districts.groupby('Urban-centric Locale [District] 2009-10')[['Total Students [Public School] 2009-10']]
topq = topquartile.quantile(q=.75)
topq.head(20)

,Total Students [Public School] 2009-10
Urban-centric Locale [District] 2009-10,
11-City: Large,34462.25
12-City: Mid-size,26544.50
13-City: Small,11804.75
21-Suburb: Large,7033.00
22-Suburb: Mid-size,6290.00
23-Suburb: Small,5624.00
31-Town: Fringe,3404.00
32-Town: Distant,3272.00
33-Town: Remote,2838.50


In [44]:
bottomquartile = districts.groupby('Urban-centric Locale [District] 2009-10')[['Total Students [Public School] 2009-10']]
bottomq = bottomquartile.quantile(q=.25)
bottomq.head(20)

,Total Students [Public School] 2009-10
Urban-centric Locale [District] 2009-10,
11-City: Large,534.00
12-City: Mid-size,2217.50
13-City: Small,4306.50
21-Suburb: Large,2192.50
22-Suburb: Mid-size,1955.00
23-Suburb: Small,1488.00
31-Town: Fringe,1367.50
32-Town: Distant,1298.50
33-Town: Remote,994.25


In [45]:
groups = []

for name, group in topquartile:
    groups.append(name)

districts['Student_Size'] = 'Medium'
for index in range(len(groups)):
    districts['Student_Size'] = np.where((districts['Urban-centric Locale [District] 2009-10']==groups[index]) & (districts['Total Students [Public School] 2009-10']>topq.iloc[index]['Total Students [Public School] 2009-10']), 'Large', districts['Student_Size'])
    districts['Student_Size'] = np.where((districts['Urban-centric Locale [District] 2009-10']==groups[index]) & (districts['Total Students [Public School] 2009-10']<=bottomq.iloc[index]['Total Students [Public School] 2009-10']), 'Small', districts['Student_Size'])

print districts['Student_Size'].value_counts()

Medium    5397
Small     2696
Large     2692
dtype: int64


Determine the top quartile and bottom quartile boundaries for free and reduced price lunch ratio.

In [46]:
topquartile = districts.groupby(['Urban-centric Locale [District] 2009-10', 'Student_Size'])[['r_lunch_free_reduced']]
topq = topquartile.quantile(q=.75)
topq.head(20)

r_lunch_free_reduced
Urban-centric Locale [District] 2009-10 Student_Size                      
11-City: Large                          Large                     0.760625
                                        Medium                    0.781110
                                        Small                     0.863215
12-City: Mid-size                       Large                     0.699413
                                        Medium                    0.686311
                                        Small                     0.786781
13-City: Small                          Large                     0.656074
                                        Medium                    0.662596
                                        Small                     0.749835
21-Suburb: Large                        Large                     0.520991
                                        Medium                    0.414711
                                        Small                     0.544477
22-Suburb: Mid-size                     Large                     0.551703
                                        Medium                    0.490063
                                        Small                     0.598169
23-Suburb: Small                        Large                     0.567036
                                        Medium                    0.473632
                                        Small                     0.585396
31-Town: Fringe                         Large                     0.487223
                                        Medium                    0.465843

In [47]:
bottomquartile = districts.groupby(['Urban-centric Locale [District] 2009-10', 'Student_Size'])[['r_lunch_free_reduced']]
bottomq = bottomquartile.quantile(q=.25)
bottomq.head(20)

r_lunch_free_reduced
Urban-centric Locale [District] 2009-10 Student_Size                      
11-City: Large                          Large                     0.516167
                                        Medium                    0.452896
                                        Small                     0.578877
12-City: Mid-size                       Large                     0.473946
                                        Medium                    0.376341
                                        Small                     0.449552
13-City: Small                          Large                     0.374962
                                        Medium                    0.354174
                                        Small                     0.385787
21-Suburb: Large                        Large                     0.182504
                                        Medium                    0.097351
                                        Small                     0.141586
22-Suburb: Mid-size                     Large                     0.263716
                                        Medium                    0.177061
                                        Small                     0.260663
23-Suburb: Small                        Large                     0.338845
                                        Medium                    0.249453
                                        Small                     0.298682
31-Town: Fringe                         Large                     0.241222
                                        Medium                    0.228426

In [48]:
groups = []

for name, group in topquartile:
    groups.append(name)

districts['Lunch_Status'] = 'Average'
for index in range(len(groups)):
    districts['Lunch_Status'] = np.where((districts['Student_Size']==groups[index][1]) & (districts['Urban-centric Locale [District] 2009-10']==groups[index][0]) & (districts['r_lunch_free_reduced']>topq.iloc[index]['r_lunch_free_reduced']), 'High', districts['Lunch_Status'])
    districts['Lunch_Status'] = np.where((districts['Student_Size']==groups[index][1]) & (districts['Urban-centric Locale [District] 2009-10']==groups[index][0]) & (districts['r_lunch_free_reduced']<=bottomq.iloc[index]['r_lunch_free_reduced']), 'Low', districts['Lunch_Status'])

print districts['Lunch_Status'].value_counts()

Average    5509
Low        2642
High       2634
dtype: int64


Check even distribution of grouped similar schools row counts.

In [49]:
districts.groupby(['Urban-centric Locale [District] 2009-10', 'Student_Size', 'Lunch_Status'])['AFGR'].count()

Urban-centric Locale [District] 2009-10  Student_Size  Lunch_Status
11-City: Large                           Large         Average          41
                                                       High             16
                                                       Low              16
                                         Medium        Average          86
                                                       High             30
                                                       Low              30
                                         Small         Average          37
                                                       High             18
                                                       Low              18
12-City: Mid-size                        Large         Average          22
                                                       High             11
                                                       Low              11
                                

In [50]:
similardistrictresults = districts.groupby(['Urban-centric Locale [District] 2009-10', 'Student_Size', 'Lunch_Status'])['AFGR'].aggregate([np.mean, np.min, np.max])
similardistrictresults.to_csv("data/finaldata/similardistrictresults.csv")
similardistrictresults.head(100)

mean  amin   amax
Urban-centric Locale [District] 2009-10 Student_Size Lunch_Status                        
11-City: Large                          Large        Average       68.821951  45.3   89.7
                                                     High          62.668750  48.2   81.9
                                                     Low           79.237500  61.0   93.7
                                        Medium       Average       68.116279  31.2  100.0
                                                     High          62.916667  30.5  100.0
                                                     Low           80.126667  21.4  100.0
                                        Small        Average       68.740541  12.9  100.0
                                                     High          68.216667   3.4  100.0
                                                     Low           75.605556  53.3  100.0
12-City: Mid-size                       Large        Average       69.931818  52.5   81.1
                                                     High          59.036364  49.6   85.2
                                                     Low           84.872727  54.0  100.0
                                        Medium       Average       78.297674  49.8  100.0
                                                     High          61.931818  43.8   82.3
                                                     Low           83.250000   0.9   99.3
                                        Small        Average       75.342308  32.4  100.0
                                                     High          52.588889  21.4  100.0
                                                     Low           74.833333  41.4  100.0
13-City: Small                          Large        Average       76.768085  62.0   94.3
                                                     High          64.504348  42.2   94.7
                                                     Low           86.179167  68.1  100.0
                                        Medium       Average       78.528421  53.3  100.0
                                                     High          62.930435  42.7   86.0
                                                     Low           90.714894  78.0  100.0
                                        Small        Average       74.256000   7.5  100.0
                                                     High          65.472727  30.2  100.0
                                                     Low           82.359091  37.9  100.0
21-Suburb: Large                        Large        Average       82.886000   2.0  100.0
                                                     High          71.431633  29.6  100.0
                                                     Low           94.068367  72.6  100.0
...                                                                      ...   ...    ...
32-Town: Distant                        Small        High          76.779688  16.0  100.0
                                                     Low           90.853125  53.3  100.0
33-Town: Remote                         Large        Average       76.486139  43.8  100.0
                                                     High          69.595918  44.6   88.7
                                                     Low           84.014000  49.5  100.0
                                        Medium       Average       81.783902  54.9  100.0
                                                     High          71.955102  44.1   98.4
                                                     Low           87.801020   5.1  100.0
                                        Small        Average       82.146154  14.8  100.0
                                                     High          73.681250  23.5  100.0
                                                     Low           88.864583   4.5  100.0
41-Rural: Fringe                        Large        Average       78.780117   5.8  100.0
                      

In [63]:
highfull = districts[['Agency ID - NCES Assigned [District] Latest available year', 'Agency Name', 'State Name [District] Latest available year', 'Urban-centric Locale [District] 2009-10', 'Student_Size', 'Lunch_Status', 'AFGR', 'i_agency_type_regional_education_services', 'i_lgo_PK', 'i_lgo_K', 'Total Number Operational Schools [Public School] 2009-10', 'i_fin_sdlc_sec', 'r_frev_ao', 'r_frev_dis', 'Total Expenditures (TOTALEXP) per Pupil (V33) [District Finance] 2009-10', 'r_stud_reg_12_W_M', 'r_stud_reg_12_W_F']]

In [64]:
lowfull = districts[['Agency ID - NCES Assigned [District] Latest available year', 'i_fin_sdlc_elem_sec', 'i_fin_sdlc_voc', 'r_stud_reg_12_W_F', 'i_lgo_PK', 'r_stud_reg_12_W_M', 'i_lgo_K', 'i_agency_type_local_school_district', 'r_frev_ao', 'r_stud_re_B', 'r_stud_912']]

In [65]:
highnoge = districts[['Agency ID - NCES Assigned [District] Latest available year', 'i_lgo_K', 'i_lgo_PK', 'i_fin_sdlc_sec', 'r_st_IA', 'Total Number Operational Schools [Public School] 2009-10', 'r_IEP', 'Total Expenditures (TOTALEXP) per Pupil (V33) [District Finance] 2009-10', 'Total Current Expenditures - Benefits (Z34) per Pupil (V33) [District Finance] 2009-10', 'r_stud_912', 'Total Revenue - Federal Sources (TFEDREV) as Percentage of Total Revenue (TOTALREV) [District Finance] 2009-10']]

In [66]:
lownoge = districts[['Agency ID - NCES Assigned [District] Latest available year', 'i_lgo_K', 'i_lgo_PK', 'r_st_TS', 'i_fin_sdlc_voc', 'Total Expenditures (TOTALEXP) per Pupil (V33) [District Finance] 2009-10', 'r_stud_912', 'r_frev_ao', 'i_ma_metropolitan', 'i_ucl_city_small', 'i_ma_micropolitan']]

In [67]:
finalvisdistrict = highfull.merge(lowfull, 'left', 'Agency ID - NCES Assigned [District] Latest available year', suffixes=('', '_LOWFULL'))
finalvisdistrict = finalvisdistrict.merge(highnoge, 'left', 'Agency ID - NCES Assigned [District] Latest available year', suffixes=('', '_HIGHNOGE'))
finalvisdistrict = finalvisdistrict.merge(lownoge, 'left', 'Agency ID - NCES Assigned [District] Latest available year', suffixes=('', '_LOWNOGE'))

In [69]:
finalvisdistrict.to_csv("data/finaldata/tableaudistricts.csv", index=False)